# Get a Certificate for accessing the CEDA Archive

We want to access CRU TS v. 4.04 gridded datasets from the [CEDA](https://catalogue.ceda.ac.uk/uuid/89e1e34ec3554dc98594a5732622bce9) database.

In [20]:
import intake
catalog_url = "master.yaml"
catalog = intake.open_catalog(catalog_url)
atmo_cat = catalog.atmosphere
list(atmo_cat)

['HadCRUT4', 'CRU_TS']

We need to get a certificate for data access to `CRU_TS`, we take the example code from [here](https://github.com/cedadev/opendap-python-example/blob/master/remote_nc_reader.py), to create a certificate. Once, the certificate is created on your machine, you can access the data. The certification has to be done only once per machine.

In [22]:
import os
import datetime
from getpass import getpass


# Import third-party libraries
from cryptography import x509
from cryptography.hazmat.backends import default_backend

from contrail.security.onlineca.client import OnlineCaClient

# Credentials defaults
DODS_FILE_CONTENTS = """HTTP.COOKIEJAR=./dods_cookies
HTTP.SSL.CERTIFICATE=./credentials.pem
HTTP.SSL.KEY=./credentials.pem
HTTP.SSL.CAPATH=./ca-trustroots
"""

DODS_FILE_PATH = os.path.expanduser('~/.dodsrc')
CERTS_DIR = os.path.expanduser('~/.certs')

if not os.path.isdir(CERTS_DIR):
    os.makedirs(CERTS_DIR)

TRUSTROOTS_DIR = os.path.join(CERTS_DIR, 'ca-trustroots')
CREDENTIALS_FILE_PATH = os.path.join(CERTS_DIR, 'credentials.pem')

TRUSTROOTS_SERVICE = 'https://slcs.ceda.ac.uk/onlineca/trustroots/'
CERT_SERVICE = 'https://slcs.ceda.ac.uk/onlineca/certificate/'


def write_dods_file_contents():

    DODS_FILE_CONTENTS = """
    HTTP.COOKIEJAR=./dods_cookies
    HTTP.SSL.CERTIFICATE={credentials_file_path}
    HTTP.SSL.KEY={credentials_file_path}
    HTTP.SSL.CAPATH={trustroots_dir}
    """.format(credentials_file_path=CREDENTIALS_FILE_PATH, trustroots_dir=TRUSTROOTS_DIR)

    with open(DODS_FILE_PATH, 'w') as dods_file:
        dods_file.write(DODS_FILE_CONTENTS)


def cert_is_valid(cert_file, min_lifetime=0):
    """
    Returns boolean - True if the certificate is in date.
    Optional argument min_lifetime is the number of seconds
    which must remain.
    :param cert_file: certificate file path.
    :param min_lifetime: minimum lifetime (seconds)
    :return: boolean
    """
    try:
        with open(cert_file, 'rb') as f:
            crt_data = f.read()
    except IOError:
        return False

    try:
        cert = x509.load_pem_x509_certificate(crt_data, default_backend())
    except ValueError:
        return False

    now = datetime.datetime.now()

    return (cert.not_valid_before <= now
            and cert.not_valid_after > now + datetime.timedelta(0, min_lifetime))
    

    
def setup_credentials(force=False):
    """
    Download and create required credentials files.
    Return True if credentials were set up.
    Return False is credentials were already set up.
    :param force: boolean
    :return: boolean
    """
    # Test for DODS_FILE and only re-get credentials if it doesn't
    # exist AND `force` is True AND certificate is in-date.
    if os.path.isfile(DODS_FILE_PATH) and not force and cert_is_valid(CREDENTIALS_FILE_PATH):
        print('[INFO] Security credentials already set up.')
        return False

    onlineca_client = OnlineCaClient()
    onlineca_client.ca_cert_dir = TRUSTROOTS_DIR

    # Set up trust roots
    trustroots = onlineca_client.get_trustroots(
        TRUSTROOTS_SERVICE,
        bootstrap=True,
        write_to_ca_cert_dir=True)
    
    username = input("CEDA username")#os.environ['CEDA_USERNAME']
    password = getpass("CEDA password")#os.environ['CEDA_PASSWORD']

    # Write certificate credentials file
    key_pair, certs = onlineca_client.get_certificate(
        username,
        password,
        CERT_SERVICE,
        pem_out_filepath=CREDENTIALS_FILE_PATH)

    # Write the dodsrc credentials file
    write_dods_file_contents()

    print('[INFO] Security credentials set up.')
    return True


Now, we can simply run the `setup_credentials`

In [24]:
setup_credentials(force=True)

/opt/anaconda3/envs/dev/lib/python3.6/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'slcs.ceda.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


CEDA username lbuntemeyer
CEDA password ··········


[INFO] Security credentials set up.


True

Great! Now we should be able to discover the `CRU_TS` dataset!

In [26]:
atmo_cat.CRU_TS.discover()

{'datashape': None,
 'dtype': None,
 'shape': None,
 'npartitions': None,
 'metadata': {'url': 'https://crudata.uea.ac.uk/cru/data/hrg',
  'doi': '10.5285/10d3e3640f004c578403419aac167d82',
  'comment': 'Access to these data is available to any registered CEDA user.',
  'catalog_dir': '/Users/lars/python/remote_climate_data/',
  'dims': {'lat': 360, 'lon': 720, 'time': 1416},
  'data_vars': {'tmp': ['lon', 'lat', 'time'], 'stn': ['lon', 'lat', 'time']},
  'coords': ('lon', 'lat', 'time'),
  'Conventions': 'CF-1.4',
  'title': 'CRU TS4.03 Mean Temperature',
  'institution': 'Data held at British Atmospheric Data Centre, RAL, UK.',
  'source': 'Run ID = 1905011326. Data generated from:tmp.1905011321.dtb',
  'history': 'Wed  1 May 2019 15:42:51 BST : User ianharris : Program makegridsauto.for called by update.for',
  'references': 'Information on the data is available at http://badc.nerc.ac.uk/data/cru/',
  'contact': 'support@ceda.ac.uk',
  'DODS_EXTRA.Unlimited_Dimension': 'time'}}

Let's access the temperature variable to check if everything works:

In [28]:
cru_tmp_ds = atmo_cat.CRU_TS.get(variable='tmp').read_chunked()
cru_tmp_ds

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720, time: 1416)
Coordinates:
  * lon      (lon) float32 -179.75 -179.25 -178.75 ... 178.75 179.25 179.75
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 1901-01-16 1901-02-15 ... 2018-12-16
Data variables:
    tmp      (time, lat, lon) float32 ...
    stn      (time, lat, lon) float64 ...
Attributes:
    Conventions:                     CF-1.4
    title:                           CRU TS4.03 Mean Temperature
    institution:                     Data held at British Atmospheric Data Ce...
    source:                          Run ID = 1905011326. Data generated from...
    history:                         Wed  1 May 2019 15:42:51 BST : User ianh...
    references:                      Information on the data is available at ...
    comment:                         Access to these data is available to any...
    contact:                         support@ceda.ac.uk
    DODS_EXTRA.Unlimited_Dimension:  time

In [29]:
cru_tmp_ds.tmp

<xarray.DataArray 'tmp' (time: 1416, lat: 360, lon: 720)>
[367027200 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -179.75 -179.25 -178.75 ... 178.75 179.25 179.75
  * lat      (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * time     (time) datetime64[ns] 1901-01-16 1901-02-15 ... 2018-12-16
Attributes:
    long_name:                   near-surface temperature
    units:                       degrees Celsius
    correlation_decay_distance:  1200.0